<a href="https://colab.research.google.com/github/Noxybot/tt_21/blob/master/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMModel,LGBMClassifier
from sklearn.metrics import precision_score, recall_score
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import time


In [4]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv",)

In [5]:
X_train = train_data
X_train = X_train.drop(X_train.columns[0], axis=1)
X_train = X_train.drop(X_train.columns[-1], axis=1)
X_train.fillna(X_train.mean(), inplace=True)
X_train.tail()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
957914,0.098979,0.20253,2192.10,0.348010,0.276430,-0.84065,2214.2,26246.0,1.061600e+15,10882.0,1.423100,-6.272100e+06,0.24397,9.12600,-0.162420,28.2710,6.3012,3.972600,28.172,918.79000,-452.11,1.2752,136.510000,1.317000,852.96,1.770100e+10,9.497900e+11,1.897300e+05,0.008593,6.3493,-0.365550,-7272.6,22593.0,0.001645,5.503200e+16,-129.81,702.35,1.6971,143.78,0.994330,...,-7.86830,-0.002791,0.9924,4.483800e+10,1.9087,-32997.0,421.910,1.625400e+09,37.24000,516.140,13820.00,10.0240,0.457150,1182.50,7.8689,0.018325,40.61100,10115.00,1.00020,4.266600e+13,1.4714,-0.005268,24.0300,4294.5,33.3480,10784.00,0.292240,0.102100,0.000843,-1.517300e+07,0.527470,-51.2970,1.7323,20.76300,14.744000,27415.0,1.2160,2.139400e+13,2612.00,0.42597
957915,0.134930,0.41360,1050.10,0.000521,-0.001246,1.76990,3519.9,862290.0,7.321400e+13,2869.0,-0.993300,2.810600e+08,0.22476,0.55288,0.088897,71.3770,5.9840,-0.043605,16.793,513.14000,133240.00,1.3749,87.166891,1.406500,516.92,1.385100e+13,2.901000e+07,2.256887e+06,0.000873,1.1054,0.926910,366280.0,1671900.0,-0.001010,1.580300e+14,1590.10,1113.70,1.4709,1625.60,1.032100,...,-7.15150,0.488000,2.9971,3.343000e+11,-3.6007,18338000.0,696.180,3.866000e+09,20.30500,91.108,-161.43,10.2940,0.495320,9312.40,55.5870,0.821190,2.97180,-50.04,0.99839,1.962400e+12,1.8432,0.259720,19.0660,824170.0,24.5430,197.20,0.067617,0.331020,-0.000175,4.326800e+09,0.099876,-41.4180,2.6986,-1.09760,-1.099800,13826.0,1.3320,1.829400e+17,3300.50,0.92974
957916,0.104660,0.47351,2415.40,0.252230,0.196970,-0.82323,3044.9,13832.0,2.693300e+12,2294.7,-1.288300,6.196500e+09,0.30436,1.30470,-0.112280,447.1800,6.9643,9.295000,30.325,7.19080,12805.00,3.6264,100.540000,-2.167300,817.29,3.374100e+11,1.155800e+10,2.491300e+05,0.011656,16.8370,0.907680,276880.0,26567.0,0.001534,1.052500e+15,1006.90,190.58,1.4557,223.83,0.913680,...,7.97440,0.485510,2.4252,2.624700e+11,366.5300,1058700.0,515.990,9.196500e+07,-0.47582,357.570,258.72,4.1402,0.286020,685.98,10.3810,1.408300,-0.33024,4043.70,0.99846,1.028400e+13,1.3445,0.307710,27.8670,5641.6,-4.1082,16378.00,0.157900,0.209340,0.000785,8.181600e+07,0.303780,-15.9670,1.6843,1.07590,16.646000,8655.1,1.1113,1.925800e+16,297.61,0.91446
957917,0.118190,0.32087,657.86,0.237290,0.260250,-0.71237,173.2,20811.0,9.711000e+14,14315.0,1.242500,5.616000e+09,0.24120,14.98600,0.012454,-4.5264,4.6367,1.998800,67.436,-0.66237,886.48,2.0113,118.050000,-0.087244,390.00,3.046200e+12,3.795800e+11,2.260300e+06,0.910290,2.9561,-0.015279,-20259.0,-18521.0,0.000433,6.432400e+16,121.00,240.97,2.0243,4520.00,-0.002366,...,-6.72220,0.015447,3.0086,3.332900e+11,146.0800,6987800.0,862.960,2.428100e+10,15.57000,132.980,8424.10,2.4245,0.305620,12132.00,388.9900,1.080900,4.54780,692.10,0.99740,4.460000e+13,1.0969,0.184940,29.4540,840350.0,305.9600,232.82,-0.007223,0.001885,-0.001871,4.914500e+09,0.001222,-2.8375,1.3006,132.50000,8.530500,3842.1,1.1941,1.248300e+16,5212.10,0.20942
957918,0.088832,0.47223,4309.40,0.017016,0.376880,4.81520,458.2,42183.0,1.805500e+13,3155.7,0.729518,4.854400e+09,0.27184,4.30380,-0.137240,30.1530,6.0480,0.975530,36.699,107.32000,165440.00,1.5196,116.500000,1.113900,1089.90,1.847900e+12,-2.451000e+10,3.561300e+05,0.015933,12.3700,1.050600,13948.0,57497.0,0.002072,2.946300e+16,-288.14,617.93,1.3991,1021.30,0.090919,...,-0.98517,0.259300,3.9379,7.810700e+10,54.7460,25941000.0,68.788,4.082500e+09,4.99360,165.780,1495.00,5.1523,-0.007529,14389.00,41.6830,0.821190,11.93900,9403.10,0.99876,6.576900e+11,2.8812,0.292870,5.9295,663230.0,18.4760,985.60,0.178027,0.072745,-0.000784,2.580200e+09,0.225230,-24.2290,2.4189,0.67939,1.748777,5901.4,1.0907,1.213800e+16,2132.30,0.38069


In [6]:
Y_train = train_data
Y_train.drop(Y_train.columns.difference(['claim']), 1, inplace=True)
#Y_train.fillna(Y_train.mean(), inplace=True)
Y_train.tail()

,claim
957914,0
957915,1
957916,0
957917,1
957918,0


In [7]:
X_test = test_data
X_test = X_test.drop(X_test.columns[0], axis=1)
X_test.fillna(X_test.mean(), inplace=True)
X_test.tail()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
493469,-0.009112,0.30819,637.64,0.778200,0.41415,-1.068500,651.22,985000.0,6.079700e+15,508.820000,-1.6804,2.833700e+09,0.274810,11.050000,-0.014169,400.14,5.7628,-0.001073,5.0498,13.743000,204.01,2.3786,-2.8254,1.02590,367.5300,1.891800e+10,9.730300e+11,6252900.0,0.898450,2.9181,0.712160,-3601.1,3.585715e+05,-0.002788,8.989500e+16,298.280,451.56,1.5524,4677.800,0.002761,...,-1.84160,0.14158,2.4004,6.137600e+09,39.73500,75812.0,2113.900,7.050100e+10,28.5400,81.661,4997.200000,17.2460,0.31933,17181.000,7.3211,1.026800,41.0360,60.576,0.99962,3.176100e+12,1.1408,0.354360,38.74500,3645.4,21.704,907.59,0.040616,0.198570,0.003273,5.782200e+07,0.249550,-17.5100,2.3325,0.112260,-5.9238,63051.287541,1.1559,8.163900e+16,857.09,1.56330
493470,0.088922,0.48265,6924.10,0.025963,0.35540,-0.870200,2514.20,18004.0,6.073500e+14,-0.008665,2.1499,5.827800e+06,0.294350,-0.004783,0.023644,596.51,7.9760,9.435700,21.0210,851.480000,14510.00,2.4662,89.3290,-0.16624,949.4100,1.270600e+12,1.358000e+11,1037100.0,-0.000373,4.4537,-0.007206,40413.0,1.143100e+06,0.000631,2.432400e+15,-158.550,244.97,1.3334,4061.900,0.904380,...,-14.27800,0.26539,1.9843,2.423700e+11,676.62000,21457.0,2091.900,1.970600e+10,31.5060,188.670,1569.200000,5.4486,0.47033,169.150,129.9900,-0.286240,1.3692,9618.300,0.99864,-3.458700e+10,1.3751,0.238700,34.74900,604250.0,-54.677,8563.40,0.073362,0.181930,0.046054,1.503400e+09,0.017970,-7.8690,1.7005,97.813000,4.3793,-2432.000000,1.0707,4.691800e+16,7497.10,0.67075
493471,0.140620,0.48475,1797.10,0.147020,0.28803,-1.407100,434.03,333050.0,2.351000e+15,33492.000000,1.2310,6.030700e+09,0.224030,0.180100,0.000926,226.66,6.7145,2.931500,13.8390,204.179272,128560.00,2.6174,104.0000,1.14650,394.1400,2.181700e+12,5.170600e+12,3314200.0,-0.002885,1.1085,1.041000,11661.0,-5.084100e+04,0.000345,2.790835e+16,-85.664,833.87,1.4733,67.176,-0.000486,...,7.77390,0.31598,1.4057,1.027400e+11,285.22000,379880.0,35.623,6.274400e+10,36.5470,115.350,10423.000000,3.3315,0.39220,4995.600,38.6460,-0.012972,23.8930,342.900,1.00040,8.181500e+11,1.4087,0.797330,13.03100,1209100.0,68.528,260.56,0.109100,0.346760,0.000079,1.920600e+09,0.013841,-5.3824,1.6827,23.846392,8.0633,2471.400000,1.1725,7.900900e+16,2904.60,0.18005
493472,0.168000,0.35176,454.79,0.164580,0.16983,0.323850,2331.20,223980.0,-2.795300e+12,93.561000,7.2319,1.175200e+09,0.129590,6.032200,0.403760,725.68,7.6410,6.632600,24.2630,40.471000,-1252.90,2.5204,92.9100,1.47880,2.6615,2.289200e+13,4.518200e+12,136650.0,-0.002484,9.6386,0.879300,507380.0,3.286400e+05,0.002744,1.273900e+17,213.210,302.92,2.0736,3682.200,0.916260,...,-0.47804,0.24513,3.0052,2.048200e+11,-0.15322,347000.0,2146.800,6.722200e+10,2.5834,451.050,6128.300000,5.2362,0.36444,594.880,71.7380,0.024923,28.5220,22863.000,0.99832,5.116900e+13,1.2432,0.057941,0.19997,144140.0,30.394,471.92,0.002906,0.197590,0.042522,3.240100e+09,0.858730,-4.2416,1.3531,155.210000,13.9630,-11.440000,1.1946,-1.770600e+14,6763.10,0.33223
493473,0.093079,0.50186,3322.40,0.201503,0.33269,0.056827,2568.70,39185.0,1.489300e+14,-318.920000,-2.3452,5.552200e+09,0.013716,6.980486,0.019480,1229.50,3.0770,-0.036249,2.0722,376.310000,884.18,2.0137,88.7720,1.13980,700.3300,8.834800e+11,3.820400e+10,3605800.0,0.094938,25.7170,0.838200,36862.0,-5.116400e+04,0.001486,-5.152300e+13,-325.980,439.93,2.6547,284.100,1.014500,...,4.33650,0.18905,2.9926,6.006600e+09,344.23000,42794.0,1928.200,7.394200e+10,69.0480,212.940,3809.778817,2.9230,0.28574,50.874,296.8300,0.821535,9.9958,324.190,1.00090,-1.136900e+10,3.1286,0.154810,30.78400,557030.0,84.958,176.55,0.258370,0.071935,-0.002442,7.436200e+07,0.889390,-13.9390,1.4480,-1.512900,-15.4360,662.460000,

In [36]:
class MyRandomForestClassifier:
  criterion='gini'
  max_depth=None
  min_samples_split=2
  min_samples_leaf=1
  min_weight_fraction_leaf=0.0
  max_features='auto'
  max_leaf_nodes=None
  min_impurity_decrease=0.0
  min_impurity_split=None

  trees = []
  def __init__(self, n_estimators=5, bootstrap=True,
               random_state=None, max_samples=None):
    self.n_estimators = n_estimators
    self.random_state = random_state
    self.max_samples = max_samples
    self.bootstrap = bootstrap
  def fit(self, X, Y):
    bootstraped_size = X.shape[0]
    if isinstance(self.max_samples, int):
      bootstraped_size = max_samples
    elif isinstance(self.max_samples, float):
      bootstraped_size = int(self.max_samples * X.shape[0])
    print("size of sample: ",  bootstraped_size)
    clf = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split,
                                 min_weight_fraction_leaf=self.min_weight_fraction_leaf,
                                 min_samples_leaf=self.min_samples_leaf, max_features=self.max_features,
                                 max_leaf_nodes=self.max_leaf_nodes, min_impurity_decrease=self.min_impurity_decrease,
                                 min_impurity_split=self.min_impurity_split)
    if self.bootstrap is True:
      print ('doint boots')
    for i in range(self.n_estimators):
      if self.bootstrap is True:
        bootstrapped_sample = X.sample(n=bootstraped_size, axis=0);
        self.trees.append(clf.fit(bootstrapped_sample, Y.iloc[bootstrapped_sample.index.to_list()]))
      else:
          self.trees.append(clf.fit(X, Y))
  def predict(self, X):
    sub_res = []
    for tree in self.trees:
      sub_res.append(tree.predict(X))
    return np.asarray(sub_res).mean(axis=0).round()

In [38]:
#TASK 1

my_forest = MyRandomForestClassifier(bootstrap=True, n_estimators=5, max_samples=0.5)

start_fit = time.time()
my_forest.fit(X_train, Y_train)
end_fit = time.time()

my_forest_res_train = my_forest.predict(X_train)
print ('my forest precision on train', precision_score(my_forest_res_train, Y_train.to_numpy()))
print ('my forest recall on train', recall_score(my_forest_res_train, Y_train.to_numpy()))
#pd.set_option('display.max_rows', None)

start_pred = time.time()
my_forest_res_test = my_forest.predict(X_test)
end_pred = time.time()

print ('my forest fit time: ', end_fit - start_fit, ', my forest predict time: ', end_pred - start_pred)

df = pd.DataFrame({'id': test_data.id, 'claim': my_forest_res_test})
df.to_csv('my_forest_result.csv', index=False)

size of sample:  478959
doint boots
my forest precision on train 0.5684156518643393
my forest recall on train 0.9035128306697779
my forest fit time:  85.12597870826721 , my forest predict time:  4.821582555770874


In [39]:
clf = RandomForestClassifier(max_depth=5, random_state=0)

start_fit = time.time()
clf.fit(X_train, Y_train.values.ravel())
end_fit = time.time()

sklearn_res_train = clf.predict(X_train)
print ('sklearn forest precision on train', precision_score(sklearn_res_train, Y_train.to_numpy()))
print ('sklearn forest recall on train', recall_score(sklearn_res_train, Y_train.to_numpy()))
#pd.set_option('display.max_rows', None)

start_pred = time.time()
sklearn_res_test = clf.predict(X_test)
end_pred = time.time()

print ('sklearn forest fit time: ', end_fit - start_fit, ', my forest predict time: ', end_pred - start_pred)

df = pd.DataFrame({'id': test_data.id, 'claim': sklearn_res_test})
df.to_csv('sklearn_forest_result.csv', index=False)

sklearn forest precision on train 0.518295760342607
sklearn forest recall on train 0.5704992854179153
sklearn forest fit time:  643.4660634994507 , my forest predict time:  3.9493911266326904


In [41]:
Lgb = LGBMClassifier(boosting_type='goss', num_leaves=300)

start_fit = time.time()
fit_model = Lgb.fit(X_train, Y_train.values.ravel())
end_fit = time.time()

LGBM_res_train = fit_model.predict(X_train)
print ('LGBM forest precision on train', precision_score(LGBM_res_train, Y_train.to_numpy()))
print ('LGBM forest recall on train', recall_score(LGBM_res_train, Y_train.to_numpy()))

start_pred = time.time()
LGBM_res_test = fit_model.predict(X_test)
end_pred = time.time()

print ('LGBM forest fit time: ', end_fit - start_fit, ', LGBM forest predict time: ', end_pred - start_pred)

df = pd.DataFrame({'id': test_data.id, 'claim': LGBM_res_test})
df.to_csv('LGBM_goss_leaves_300_forest_result.csv', index=False)

LGBM forest precision on train 0.6961896484927175
LGBM forest recall on train 0.7402168273908794
LGBM forest fit time:  121.04174423217773 , LGBM forest predict time:  5.781458139419556


In [ ]:
LGBM_forest_result.csv
0.55316 - private
0.55481 - public

sklearn_forest_result.csv

0.54839 - private
0.55046 - public

my_forest_result.csv

0.50900 - private
0.50934 - public